In [5]:
import sys
sys.path.append('../../..')
import Dataset

import numpy as np

In [2]:
class PersonInWifiDataset(Dataset.CSI_Dataset):
    def __init__(self, 
                 data_root: str, split: str = 'train', 
                 crop_size: tuple = (192, 256), interpolation: float = -1,
                 amp_offset: float = 60000, pha_offset: float = 28000):
        super().__init__(data_root, split, crop_size, interpolation, amp_offset, pha_offset)

    def __getitem__(self, idx):
        choose = np.random.random()
        data_path = self.data_list[idx][0]
        env = data_path.split('/')[0]
        mask_path = data_path.replace('npy', 'img') + '/' + self.data_list[idx][2]
        amp, pha, mask = self.get_data(mask_path, data_path, self.data_list[idx][1:])
        label = 0

        if self.split == 'train':
            if choose < 0.5:
                idx = np.random.randint(len(self.data_struct[env]))
                data_path2 = self.data_struct[env][idx]
                mask_path2 = data_path2[0].replace('npy', 'img') + '/' + data_path2[2]
                amp2, pha2, mask2 = self.get_data(mask_path2, data_path2[0], data_path2[1:])
                label = 1

            if choose >= 0.5:
                data_struct = self.data_struct.copy()
                data_struct.pop(env)
                env2 = np.random.choice(list(data_struct.keys()))
                idx = np.random.randint(len(data_struct[env2]))
                data_path2 = data_struct[env2][idx]
                mask_path2 = data_path2[0].replace('npy', 'img') + '/' + data_path2[2]
                amp2, pha2, mask2 = self.get_data(mask_path2, data_path2[0], data_path2[1:])
                label = -1
            
            return [[amp, pha, mask], [amp2, pha2, mask2]], torch.tensor(label)
        
        else:
            return amp, pha, mask

In [3]:
dataset = PersonInWifiDataset(data_root='/root/CSI_Dataset')

In [6]:
dataset[0]

FileNotFoundError: [Errno 2] No such file or directory: '/root/CSI_Dataset/E/v_mask.png'